In [1]:
import numpy as np

In [27]:
a = np.ones((3,3,2,2,2))

In [28]:
a[0,1,0,1,0] = 100

In [29]:
b = a + np.arange(9).reshape((3,3,1,1,1))

In [30]:
b[:,:,0,1,0]

array([[  1., 101.,   3.],
       [  4.,   5.,   6.],
       [  7.,   8.,   9.]])

In [38]:
b.shape

(3, 3, 2, 2, 2)

In [37]:
b.mean(axis=(2,3,4))

array([[ 1.   , 14.375,  3.   ],
       [ 4.   ,  5.   ,  6.   ],
       [ 7.   ,  8.   ,  9.   ]])

In [39]:
const = 10

In [40]:
f = lambda x : x + const

In [42]:
const = 10000

In [43]:
f(2)

10002

In [24]:
tmp[:,:,np.newaxis,np.newaxis,np.newaxis].shape

(3, 3, 1, 1, 1)

In [22]:
np.newaxis

In [1]:
%run hyper-elasticity_debug.py

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[1. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
  4.98e-02
  1.95e-02
  2.45e-03
  9.92e-05
  1.96e-07


In [2]:
whos

Variable       Type        Data/Info
------------------------------------
DbarF          ndarray     3x3x5x5x5: 1125 elems, type `float64`, 9000 bytes
F              ndarray     3x3x5x5x5: 1125 elems, type `float64`, 9000 bytes
Fn             float64     22.360679774997898
G              function    <function <lambda> at 0x7f18cbff63b0>
G_K_dF         function    <function <lambda> at 0x7f18cbff64d0>
Ghat4          ndarray     3x3x3x3x5x5x5: 10125 elems, type `float64`, 81000 bytes
I              ndarray     3x3x5x5x5: 1125 elems, type `float64`, 9000 bytes
I4             ndarray     3x3x3x3x5x5x5: 10125 elems, type `float64`, 81000 bytes
I4rt           ndarray     3x3x3x3x5x5x5: 10125 elems, type `float64`, 81000 bytes
I4s            ndarray     3x3x3x3x5x5x5: 10125 elems, type `float64`, 81000 bytes
II             ndarray     3x3x3x3x5x5x5: 10125 elems, type `float64`, 81000 bytes
K              ndarray     5x5x5: 125 elems, type `float64`, 1000 bytes
K4             ndarray     3x3x3

In [3]:
tmp_1 = fft(F)
tmp_1.shape

(3, 3, 5, 5, 5)

In [4]:
Ghat4.shape

(3, 3, 3, 3, 5, 5, 5)

In [61]:
F_rand = np.random.rand(*F.shape)

In [8]:
np.linalg.norm(F_rand)

19.00174854076234

In [13]:
# Function to compute the curl of a vector field component
def compute_curl(U, V, W):
    curl_x = np.gradient(W, axis=1) - np.gradient(V, axis=2)
    curl_y = np.gradient(U, axis=2) - np.gradient(W, axis=0)
    curl_z = np.gradient(V, axis=0) - np.gradient(U, axis=1)
    return np.array([curl_x, curl_y, curl_z])

def curl_F(F): # F in shape [3,3,nx,ny,nz]
    # Initialize an array to store the curl of the tensor field
    curl_F = np.zeros((3, 3, *F.shape[2:]))

    # Compute the curl for each component of the stress tensor
    for i in range(3):
        for j in range(3):
            curl_F[i, j] = compute_curl(F[(i+1)%3, (j+1)%3], F[(i+2)%3, (j+1)%3], F[(i+2)%3, (j+2)%3])

    # curl_F is the resulting field
    return curl_F

In [59]:
e_ijk = lambda i,j,k: (i-j)*(j-k)*(k-i)/2
F_jk_i = lambda F,j,k,i: np.gradient(F[j,k,:,:,:],axis=i)
curl_F_mk = lambda F,m,k: np.sum([e_ijk(i,j,m)*F_jk_i(F,j,k,i) for i,j in itertools.product(range(3),repeat=2)],axis=0)
curl_F = lambda F: np.array([curl_F_mk(F,m,k) for m,k in itertools.product(range(3),repeat=2)]).reshape(F.shape)

try to use DAMASK grid_filters to check consistency

In [65]:
cF = curl_F(F_rand).shape


In [66]:
np.linalg.norm(cF)

9.643650760992955

In [67]:
G_x = lambda F_rand: G(F_rand).reshape([3,3,*shape])
GGG_F_rand = G_x(G_x(G_x(G_x(G_x(F_rand)))))
c_GGG_F = curl_F(GGG_F_rand)
np.linalg.norm(c_GGG_F)

8.189518702692673

In [70]:
GFr = G_x(F_rand)
cGFr = curl_F(GFr)
np.linalg.norm(cGFr)

8.189518702692672

In [24]:
np.reshape(G_F_rand,[3,3,*shape]).shape

(3, 3, 5, 5, 5)

In [17]:
# Function to compute the 2D curl-like operation for a component
def compute_curl_2d(U, V):
    curl = np.gradient(V, axis=2) - np.gradient(U, axis=3)
    return curl

def curl_F_2(F):
    # Initialize an array to store the curl of the tensor field
    curl_F = np.zeros((3, 3, *F.shape[2:]))

    # Compute the curl for each component of the stress tensor
    # Note: The last dimension is squeezed since nz = 1
    for i in range(3):
        for j in range(3):
            U = F[(i+1)%3, (j+1)%3, :, :, 0]
            V = F[(i+2)%3, (j+1)%3, :, :, 0]
            curl_F[i, j, :, :, 0] = compute_curl_2d(U, V)

    # curl_F is the resulting field
    return curl_F

In [20]:
tmp_2 = ddot42(Ghat4, tmp_1)
tmp_2.shape

(3, 3, 3, 3, 1)

In [21]:
tmp_3 = ifft(tmp_2)
tmp_3.shape

(3, 3, 3, 3, 1)

In [22]:
tmp_3.reshape(-1).shape

(81,)

In [24]:
%less hyper-elasticity_debug.py

In [25]:
who

DbarF	 F	 Fn	 G	 G_K_dF	 Ghat4	 I	 I4	 I4rt	 
I4s	 II	 K	 K4	 K_dF	 Nx	 Ny	 Nz	 P	 
Q	 Z	 b	 constitutive	 dFm	 ddot42	 ddot44	 delta	 dot11	 
dot22	 dot24	 dot42	 dyad22	 fft	 freq	 i	 ifft	 iiter	 
itertools	 j	 l	 m	 mu	 ndim	 norm	 np	 param	 
phase	 q	 shape	 sp	 tmp_1	 tmp_2	 tmp_3	 trans2	 x	 



In [29]:
mu.shape

(3, 3, 1)

In [ ]:
q